In [3]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(spark.sparkContext, '43285b0e-9e6e-4aad-8a96-3c40cf6f8874', 'p-feff31088fbf9bf46cb5e9470d87ed7c2432902c')
pc = project.project_context


In [4]:

import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'service_id': '    'iam_service_endpoint': 'https://iam.ng.bluemix.net/oidc/token',
    'api_key': '}

configuration_name = 'os_c63666ea1c9d403e811d4c86742ad623_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df_data_1 = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('train.csv', 'm5forecastingaccuracy-donotdelete-pr-5fybfotj7xtmkk'), inferSchema= 'true')
#df_data_1.take(5)


In [5]:

df_store = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('store.csv', 'm5forecastingaccuracy-donotdelete-pr-5fybfotj7xtmkk'), inferSchema= 'true')
df_store.take(1)


[Row(Store=1, StoreType='c', Assortment='a', CompetitionDistance=1270, CompetitionOpenSinceMonth=9, CompetitionOpenSinceYear=2008, Promo2=0, Promo2SinceWeek=None, Promo2SinceYear=None, PromoInterval=None)]

In [6]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_c63666ea1c9d403e811d4c86742ad623 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='-ynWC7Y9F5nKlpdFO21VLMfHYNAPb03HUbUqKxnp07WH',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_c63666ea1c9d403e811d4c86742ad623.get_object(Bucket='m5forecastingaccuracy-donotdelete-pr-5fybfotj7xtmkk',Key='sample_submission.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

sample_sub = pd.read_csv(body)
sample_sub.head(1)


,Id,Sales
0,1,0


In [7]:

df_test = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('test.csv', 'm5forecastingaccuracy-donotdelete-pr-5fybfotj7xtmkk'), inferSchema= 'true')
df_test.take(1)


[Row(Id=1, Store=1, DayOfWeek=4, Date=datetime.datetime(2015, 9, 17, 0, 0), Open=1, Promo=1, StateHoliday='0', SchoolHoliday=0)]

In [8]:
df_store.printSchema()

root
 |-- Store: integer (nullable = true)
 |-- StoreType: string (nullable = true)
 |-- Assortment: string (nullable = true)
 |-- CompetitionDistance: integer (nullable = true)
 |-- CompetitionOpenSinceMonth: integer (nullable = true)
 |-- CompetitionOpenSinceYear: integer (nullable = true)
 |-- Promo2: integer (nullable = true)
 |-- Promo2SinceWeek: integer (nullable = true)
 |-- Promo2SinceYear: integer (nullable = true)
 |-- PromoInterval: string (nullable = true)



In [9]:
#Join the data from df_store with the sales data in df_data_1

df_j = df_data_1.join(df_store, "Store")
df_j_test = df_test.join(df_store, "Store")

print('Number of records in df_j:', df_j.count())
print('Number of records in df_j_test:', df_j_test.count())

Number of records in df_j: 1017209
Number of records in df_j_test: 41088


In [10]:
df_j.printSchema()

root
 |-- Store: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Sales: integer (nullable = true)
 |-- Customers: integer (nullable = true)
 |-- Open: integer (nullable = true)
 |-- Promo: integer (nullable = true)
 |-- StateHoliday: string (nullable = true)
 |-- SchoolHoliday: integer (nullable = true)
 |-- StoreType: string (nullable = true)
 |-- Assortment: string (nullable = true)
 |-- CompetitionDistance: integer (nullable = true)
 |-- CompetitionOpenSinceMonth: integer (nullable = true)
 |-- CompetitionOpenSinceYear: integer (nullable = true)
 |-- Promo2: integer (nullable = true)
 |-- Promo2SinceWeek: integer (nullable = true)
 |-- Promo2SinceYear: integer (nullable = true)
 |-- PromoInterval: string (nullable = true)



In [11]:
df_j_test.printSchema()

root
 |-- Store: integer (nullable = true)
 |-- Id: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Open: integer (nullable = true)
 |-- Promo: integer (nullable = true)
 |-- StateHoliday: string (nullable = true)
 |-- SchoolHoliday: integer (nullable = true)
 |-- StoreType: string (nullable = true)
 |-- Assortment: string (nullable = true)
 |-- CompetitionDistance: integer (nullable = true)
 |-- CompetitionOpenSinceMonth: integer (nullable = true)
 |-- CompetitionOpenSinceYear: integer (nullable = true)
 |-- Promo2: integer (nullable = true)
 |-- Promo2SinceWeek: integer (nullable = true)
 |-- Promo2SinceYear: integer (nullable = true)
 |-- PromoInterval: string (nullable = true)



In [12]:
#Calculate days since competition opened

import pyspark.sql.functions as F

cols_to_drop = ['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'CompOpenSince']

df_j = df_j.withColumn("CompOpenSince", F.to_date(F.concat_ws("-", "CompetitionOpenSinceYear", "CompetitionOpenSinceMonth")))
df_j_test = df_j_test.withColumn("CompOpenSince", F.to_date(F.concat_ws("-", "CompetitionOpenSinceYear", "CompetitionOpenSinceMonth")))

df_j = df_j.withColumn('DaysSinceCompOpen', F.datediff(df_j.Date, df_j.CompOpenSince))
df_j_test = df_j_test.withColumn('DaysSinceCompOpen', F.datediff(df_j_test.Date, df_j_test.CompOpenSince))

df_j = df_j.drop(*cols_to_drop)
df_j_test = df_j_test.drop(*cols_to_drop)

df_j.take(1)


[Row(Store=1, DayOfWeek=5, Date=datetime.datetime(2015, 7, 31, 0, 0), Sales=5263, Customers=555, Open=1, Promo=1, StateHoliday='0', SchoolHoliday=1, StoreType='c', Assortment='a', CompetitionDistance=1270, Promo2=0, Promo2SinceWeek=None, Promo2SinceYear=None, PromoInterval=None, DaysSinceCompOpen=2524)]

In [13]:
# Extract year, month and week from date

df_j = df_j.withColumn('Year',F.year('Date'))
df_j = df_j.withColumn('Month',F.month('Date'))
df_j = df_j.withColumn('Week',F.weekofyear('Date'))

df_j_test = df_j_test.withColumn('Year',F.year('Date'))
df_j_test = df_j_test.withColumn('Month',F.month('Date'))
df_j_test = df_j_test.withColumn('Week',F.weekofyear('Date'))







In [14]:
df_j.printSchema()

root
 |-- Store: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Sales: integer (nullable = true)
 |-- Customers: integer (nullable = true)
 |-- Open: integer (nullable = true)
 |-- Promo: integer (nullable = true)
 |-- StateHoliday: string (nullable = true)
 |-- SchoolHoliday: integer (nullable = true)
 |-- StoreType: string (nullable = true)
 |-- Assortment: string (nullable = true)
 |-- CompetitionDistance: integer (nullable = true)
 |-- Promo2: integer (nullable = true)
 |-- Promo2SinceWeek: integer (nullable = true)
 |-- Promo2SinceYear: integer (nullable = true)
 |-- PromoInterval: string (nullable = true)
 |-- DaysSinceCompOpen: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Week: integer (nullable = true)



In [15]:
# Change datatype of some columns with wrong dtype

cols_to_double = ['DaysSinceCompOpen', 'CompetitionDistance']

for col in cols_to_double:
    df_j = df_j.withColumn(col, df_j[col].cast('double'))
    df_j_test = df_j_test.withColumn(col, df_j_test[col].cast('double'))

df_j = df_j.withColumn('Sales', df_j['Sales'].cast('double'))



In [16]:
def count_missings(spark_df,sort=True):
    df = spark_df.select([F.count(F.when(F.isnan(c) | F.isnull(c), c)).alias(c) for (c,c_type) in spark_df.dtypes if c_type not in ('timestamp', 'string', 'date')]).toPandas()

    if len(df) == 0:
        print("There are no any missing values!")
        return None

    if sort:
        df = df.rename(index={0: 'count'}).T.sort_values("count",ascending=False)
        return df[(df['count'] > 0)]

    return df
count_missings(df_j, sort=True)


,count
Promo2SinceWeek,508031
Promo2SinceYear,508031
DaysSinceCompOpen,323348
CompetitionDistance,2642


In [17]:
# Extract the maximum distance to a competitor store

max_comp_distance = df_j.agg({'CompetitionDistance': 'max'}).collect()[0][0]
print('Max CompetitionDistance:', max_comp_distance)


Max CompetitionDistance: 75860.0


In [18]:
# Fill in missing values

# For DaysSinceCompOpen fill in 0 as the missing value
df_j = df_j.fillna(0, subset='DaysSinceCompOpen')
df_j_test = df_j_test.fillna(0, subset='DaysSinceCompOpen')

# For CompetitionDistance assume there is no competition store closer than the maximum competition distance + 1
df_j = df_j.fillna(max_comp_distance+1, subset='CompetitionDistance')
df_j_test = df_j_test.fillna(max_comp_distance+1, subset='CompetitionDistance')

# Fill in nulls with 0
cols_for_pred = ['PromoInterval', 'Assortment', 'StoreType', 'StateHoliday', 'Store', 'DayOfWeek', 'Open', 'Promo', 'SchoolHoliday', 'CompetitionDistance', 'Promo2', 'DaysSinceCompOpen']

df_test = df_test.fillna(0, subset=cols_for_pred)
df_j_test = df_j_test.fillna(0, subset=cols_for_pred)






In [19]:
df_j.select(F.countDistinct("DayOfWeek")).show()


+-------------------------+
|count(DISTINCT DayOfWeek)|
+-------------------------+
|                        7|
+-------------------------+



In [21]:
# take a small sample of df_j to build up a model
df_js = df_j.sample(False, 0.0001, 42)

In [22]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoderEstimator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

cat_cols = ['PromoInterval', 'Assortment', 'StoreType', 'StateHoliday']
num_cols = ['Store', 'DayOfWeek', 'Open', 'Promo', 'SchoolHoliday', 'CompetitionDistance', 'Promo2', 'DaysSinceCompOpen', 'Year', 'Month', 'Week']

cols_for_model = num_cols + cat_cols + ['Sales']

df_j_red = df_j[cols_for_model]

stages = []

# Convert categorical strings to index values
for cat_col in cat_cols:
    indexer = StringIndexer(inputCol=cat_col, outputCol= cat_col +'_idx', handleInvalid='keep')
    onehot = OneHotEncoderEstimator(inputCols=[indexer.getOutputCol()], outputCols= [cat_col + '_dummy'])
    stages += [indexer, onehot]

ohecols = [col + '_dummy' for col in cat_cols]
    
# Assemble predictors into a single column 
assembler = VectorAssembler(inputCols=num_cols + ohecols, outputCol='features')

stages += [assembler]

s_train, s_test = df_j_red.randomSplit([0.8, 0.2], seed=1)

# Print out columns to be used in prediction
print(s_train.columns)

['Store', 'DayOfWeek', 'Open', 'Promo', 'SchoolHoliday', 'CompetitionDistance', 'Promo2', 'DaysSinceCompOpen', 'Year', 'Month', 'Week', 'PromoInterval', 'Assortment', 'StoreType', 'StateHoliday', 'Sales']


In [23]:
# Create a Gradient boosted tree regressor
gbt = GBTRegressor(featuresCol='features',
                           labelCol='Sales',
                           seed=1,
                           stepSize = 0.05,
                           maxIter = 100,
                           maxDepth = 4
                           )


stages += [gbt]


# Create a pipeline
pipeline = Pipeline(stages=stages)

# Create the evaluator, evaluate on Root mean square error
pip_evaluator = RegressionEvaluator(
    labelCol='Sales', predictionCol='prediction', metricName='rmse')





In [28]:
def gbt_with_cross_validation():
    # Create a parameter grid to try different parameters
    paramGrid = (ParamGridBuilder()
                 .addGrid(gbt.maxDepth, [4, 5, 6])
                 .build())



    # Use cross validation to find the best model
    cv = CrossValidator(estimator=pipeline, estimatorParamMaps=paramGrid, evaluator=pip_evaluator, numFolds=5, seed=1)

    cvModel = cv.fit(s_train)

    # Get the best model from cross validation
    best_model = cvModel.bestModel

    # Look at the stages in the best model
    print(best_model.stages)

    # select last stage (the GBTRegressor) of the best model and see info regarding which depth the model has chosen, this can be used on any parameter
    print(best_model.stages[-1].explainParam('maxDepth'))

    # Generate predictions on testing data using the best model then calculate RMSE
    predictions = best_model.transform(s_test)
    rmse = pip_evaluator.evaluate(predictions)

    print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
    
    return cvModel.bestModel

model = gbt_with_cross_validation()

[StringIndexer_a7324f6e293b, OneHotEncoderEstimator_d27478aff88f, StringIndexer_4da27289da9a, OneHotEncoderEstimator_dd2d4d600659, StringIndexer_4bb8121ed221, OneHotEncoderEstimator_29f067d30cf3, StringIndexer_9beddf6a1ca9, OneHotEncoderEstimator_7a272c173783, VectorAssembler_bde647bbcc1d, GBTRegressionModel (uid=GBTRegressor_b2bdfa6d1497) with 100 trees]
maxDepth: Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. (default: 5, current: 4)
Root Mean Squared Error (RMSE) on test data = 3017.22


In [24]:
def gbt_model():
    g_model = pipeline.fit(s_train)
    
    predictions = g_model.transform(s_test)
    rmse = pip_evaluator.evaluate(predictions)

    print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
    
    return g_model



In [25]:
model = gbt_model()

Root Mean Squared Error (RMSE) on test data = 2150.58


In [26]:
# Choose the columns
df_j_test_red = df_j_test[num_cols + cat_cols]
#df_test = df_test.fillna(0, subset=columns_num)

# Make prediction
kaggle_pred = model.transform(df_j_test_red)



In [27]:
kp_pandas = kaggle_pred.select('prediction').toPandas()
ki_pandas = df_j_test.select('Id').toPandas()
kp_pandas['Id'] = ki_pandas['Id']
kp_pandas['Sales'] = kp_pandas['prediction']
kp_pandas = kp_pandas.drop(columns=['prediction'])

In [28]:
kp_pandas.head()


,Id,Sales
0,1,7185.401230
1,2,7056.834693
2,3,8653.083220
3,4,7362.337956
4,5,8480.978733


In [29]:
project.save_data("submission13.csv", kp_pandas.to_csv(index=False))


{'file_name': 'submission13.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'm5forecastingaccuracy-donotdelete-pr-5fybfotj7xtmkk',
 'asset_id': 'ab60918a-f03c-40c2-b77c-6a3bd58b44f3'}